In [1]:
import json

import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
import os
from bert_util import bert_tokenize_data, tensor_train_test_split, train_bert_model, model_predict, get_data_loader, \
    calculate_accuracy
from util import get_dataframe_from_json

os.environ["USE_TF"] = "0"

/home/emiliano/Desktop/Università/Human Language Technologies/Project/HLT Project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-20 11:49:12.346902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-20 11:49:12.354106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747734552.362872   32772 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered


In [2]:
hld = get_dataframe_from_json('./hand_labeled/hand_labelled_dataset.json')
hld

,turn,utterance,emotion,act,hat
0,0,I'm so angry . I feel like killing someone .,anger,inform,red
1,1,Calm down . __eou__,no_emotion,inform,red
2,0,I was just about to go to bed when the telepho...,no_emotion,inform,white
3,1,Who was it ?,no_emotion,question,white
4,2,Kate . She said she was too excited to go to s...,no_emotion,inform,white
...,...,...,...,...,...
1012,6,I want to live abroad and learn to speak a dif...,no_emotion,inform,white
1013,7,I'm really sorry . But I understand .,no_emotion,commissive,white
1014,8,"Thank you , manager . __eou__",no_emotion,inform,white
1015,0,I fired Mr . Li today .,no_emotion,inform,white


In [3]:
hat_map = {
    0: "red",
    1: "white",
    2: "black",
    3: "yellow",
    4: "green",
}

reverse_hat_map = {v: k for k, v in hat_map.items()}
hld['hat'] = hld['hat'].apply(lambda x: reverse_hat_map[x])

In [4]:
hld
# Load datasetdf = pd.read_csv("tweets_annotation.csv")  # must have 'text' and 'score' columns

,turn,utterance,emotion,act,hat
0,0,I'm so angry . I feel like killing someone .,anger,inform,0
1,1,Calm down . __eou__,no_emotion,inform,0
2,0,I was just about to go to bed when the telepho...,no_emotion,inform,1
3,1,Who was it ?,no_emotion,question,1
4,2,Kate . She said she was too excited to go to s...,no_emotion,inform,1
...,...,...,...,...,...
1012,6,I want to live abroad and learn to speak a dif...,no_emotion,inform,1
1013,7,I'm really sorry . But I understand .,no_emotion,commissive,1
1014,8,"Thank you , manager . __eou__",no_emotion,inform,1
1015,0,I fired Mr . Li today .,no_emotion,inform,1


In [5]:
# import train test split
from sklearn.model_selection import train_test_split
# split the data into train and test
train_df, test_df = train_test_split(hld, test_size=0.2, random_state=42, stratify=hld['hat'])


In [6]:
token_ids, attention_masks = bert_tokenize_data(train_df['utterance'].values)
train_dataloader, val_dataloader = tensor_train_test_split(torch.tensor(train_df['hat'].values), token_ids, attention_masks, test_size=0.1)

In [7]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
epochs = 20
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()
num_training_steps = epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

model = train_bert_model(model, optimizer, scheduler, train_dataloader, val_dataloader, epochs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-------------------- Epoch 1 --------------------

Training:
---------
Start Time:       2025-05-20 11:49:18.864135
Average Loss:     1.2349173627469852
Time Taken:       0:00:07.859731

Validation:
---------
Start Time:       2025-05-20 11:49:26.724293
Average Loss:     1.38907498663122
Average Accuracy: 0.45454545454545453
Time Taken:       0:00:00.242029

-------------------- Epoch 2 --------------------

Training:
---------
Start Time:       2025-05-20 11:49:26.966701
Average Loss:     0.950564538979012
Time Taken:       0:00:07.335843

Validation:
---------
Start Time:       2025-05-20 11:49:34.303693
Average Loss:     1.1512981544841419
Average Accuracy: 0.5795454545454546
Time Taken:       0:00:00.237203

-------------------- Epoch 3 --------------------

Training:
---------
Start Time:       2025-05-20 11:49:34.541569
Average Loss:     0.6122964176794757
Time Taken:       0:00:07.291063

Validation:
---------
Start Time:       2025-05-20 11:49:41.833052
Average Loss:     1.2775

In [8]:
test_texts = test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = test_df['hat'].values.flatten()
accuracy = np.sum(preds == labels_flat) / len(labels_flat)
accuracy

np.float64(0.553921568627451)

In [9]:
test_texts = [
"We do not want to place ourselves in an inferior position . We are a good , small company---that's why you are interested .",
    "We're having trouble with Bob .",
    "Oh , I think I should try to read one . __eou__",
    "To be frank , a letter of credit would increase the cost of my import . When I open a letter of credit with a bank , I have to pay a deposit . That will tie up my money and add to my cost .",
    "I'm very glad to hear that .",
    "OK ... OK ... Anyway . I will treat you as my best friend . I hope you can do the same .",
    "I ’ m glad you are enjoying yourself ."
]
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=3, shuffle=False)
preds, confidences = model_predict(model, dl)
preds, confidences

([2, 2, 4, 2, 3, 4, 3],
 [0.9987348914146423,
  0.9988678693771362,
  0.9978197813034058,
  0.9986592531204224,
  0.9983394145965576,
  0.998264729976654,
  0.9989126920700073])

In [10]:
test_df

,turn,utterance,emotion,act,hat
299,0,My father ’ s angry face always makes my blood...,fear,inform,1
319,1,What are you talking about ?,no_emotion,question,1
688,4,I couldn ’ t agree more . That ’ s an ideal pl...,happiness,inform,0
665,0,How's your girlfriend ?,no_emotion,question,1
955,7,"I hope to make 4,000 yuan a month for supporti...",no_emotion,directive,1
...,...,...,...,...,...
890,4,What kinds of channels can you use in e-commer...,no_emotion,question,1
340,5,That's quite a compliment coming from you . __...,happiness,inform,1
514,5,Why not now ? I'm hungry . __eou__,happiness,inform,1
326,0,He has come to life in the end .,no_emotion,inform,1
